In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from pyspark.sql import functions as F  #filtering
import geopandas as gpd
import folium
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# starting a Spark session
spark = (
    SparkSession.builder.appName('Parkres Further Analysis')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

# Read the dataset from a CSV file using PySpark
parkres = spark.read.csv('../data/curated/parkres/parkres.csv', header=True, inferSchema=True)

# Drop the extra index column (_c0) if it exists
parkres = parkres.drop('_c0')

# Show the first few rows of the dataset to confirm
parkres.show(5)

your 131072x1 screen size is bogus. expect trouble
24/09/25 21:01:35 WARN Utils: Your hostname, LAPTOP-KOQUIUN resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/25 21:01:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 21:01:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/25 21:01:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+--------------------+--------------------+--------+--------------------+
|                name|            sa2_name|postcode|            geometry|
+--------------------+--------------------+--------+--------------------+
|Lilydale-Warburto...|        Yarra Valley|    3139|POLYGON ((1034153...|
|Nangana Bushland ...|        Yarra Valley|    3139|POLYGON ((1022203...|
|Nillumbik G139 Bu...|Wattle Glen - Dia...|    3089|POLYGON ((989912....|
|Lilydale-Warburto...|Lilydale - Coldst...|    3140|POLYGON ((1005216...|
|Plenty Gorge Park...|  Plenty - Yarrambat|    3088|POLYGON ((983018....|
+--------------------+--------------------+--------+--------------------+
only showing top 5 rows



In [3]:
# read the domain parquet dataset
domain = spark.read.parquet('../data/curated/domain_data')
domain.limit(5)

24/09/25 21:01:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


url,price,address,property_type,latitude,longitude,Beds,Baths,Parking,bond,extracted_price,geometry,sa2_code,sa2_name,chg_flag,chg_lbl,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,areasqkm,loci_uri,__index_level_0__
https://www.domai...,"$1,400.00","10 Allara Court, ...",Townhouse,-37.77427300000001,145.1811258,4.0,3.0,2.0,9125.0,1400.0,[01 01 00 00 00 C...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,0
https://www.domai...,$750 per week,"7 Pine Ridge, Don...",House,-37.7912513,145.1756489,4.0,2.0,0.0,3259.0,750.0,[01 01 00 00 00 8...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,1
https://www.domai...,$1300 per week,"20 Mulsanne Way, ...",House,-37.7972323,145.1812636,5.0,2.0,2.0,5649.0,1300.0,[01 01 00 00 00 9...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,2
https://www.domai...,$825pw / $3585pcm,3 Monterey Cresce...,House,-37.792402,145.1743233,3.0,1.0,5.0,3585.0,825.0,[01 01 00 00 00 C...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,3
https://www.domai...,$680.00,3/49 Leslie Stree...,Townhouse,-37.7810117,145.180705,3.0,2.0,2.0,2955.0,680.0,[01 01 00 00 00 2...,211021261,Donvale - Park Or...,0,No change,21102,Manningham - East,211,Melbourne - Outer...,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.dat...,4


In [4]:
from pyspark.sql import functions as F

# Check for null values in each column
domain.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in domain.columns]).show()

+---+-----+-------+-------------+--------+---------+----+-----+-------+----+---------------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-----------------+
|url|price|address|property_type|latitude|longitude|Beds|Baths|Parking|bond|extracted_price|geometry|sa2_code|sa2_name|chg_flag|chg_lbl|sa3_code|sa3_name|sa4_code|sa4_name|gcc_code|gcc_name|ste_code|ste_name|aus_code|aus_name|areasqkm|loci_uri|__index_level_0__|
+---+-----+-------+-------------+--------+---------+----+-----+-------+----+---------------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-----------------+
|  0|    0|      0|            0|       0|        0|   0|    0|      0|1199|              0|       0|       0|       0|       0|      0|       0|       0|       0|       0|       0|       0|       0|       0|   

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, StringType
from shapely import wkt
from pyspark.sql.functions import udf
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance between two points (Haversine formula)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# UDF to convert WKT geometry to centroid coordinates
def get_centroid(geometry):
    try:
        shape = wkt.loads(geometry)
        centroid = shape.centroid
        return f"{centroid.y},{centroid.x}"
    except:
        return None

# Register UDFs
distance_udf = udf(calculate_distance, DoubleType())
centroid_udf = udf(get_centroid, StringType())

# Prepare park data
parkres = parkres.withColumnRenamed("geometry", "park_geometry")
parkres = parkres.withColumn("park_centroid", centroid_udf(F.col("park_geometry")))
parkres = parkres.withColumn("park_centroid_lat", F.split(F.col("park_centroid"), ",")[0].cast(DoubleType()))
parkres = parkres.withColumn("park_centroid_lon", F.split(F.col("park_centroid"), ",")[1].cast(DoubleType()))

# Prepare domain data
domain = domain.withColumnRenamed("latitude", "prop_lat")
domain = domain.withColumnRenamed("longitude", "prop_lon")
domain = domain.withColumnRenamed("sa2_name", "property_sa2_name")

# Cross join to calculate the distance between each property and every park
result = domain.crossJoin(F.broadcast(parkres))

# Calculate distances (km) between properties and park centroids
result = result.withColumn("distance", 
    distance_udf(F.col("prop_lat"), F.col("prop_lon"), 
                 F.col("park_centroid_lat"), F.col("park_centroid_lon")))
result.orderBy("distance").show(5)

+--------------------+----------------+--------------------+--------------------+-----------+-----------+----+-----+-------+------+---------------+--------------------+---------+-----------------+--------+---------+--------+------------+--------+--------------------+--------+-----------------+--------+--------+--------+---------+--------+--------------------+-----------------+--------------------+--------------------+--------+--------------------+--------------------+------------------+-----------------+------------------+
|                 url|           price|             address|       property_type|   prop_lat|   prop_lon|Beds|Baths|Parking|  bond|extracted_price|            geometry| sa2_code|property_sa2_name|chg_flag|  chg_lbl|sa3_code|    sa3_name|sa4_code|            sa4_name|gcc_code|         gcc_name|ste_code|ste_name|aus_code| aus_name|areasqkm|            loci_uri|__index_level_0__|                name|            sa2_name|postcode|       park_geometry|       park_centroid

In [6]:
# Find the nearest park for each property
window_spec = Window.partitionBy("__index_level_0__").orderBy("distance")
nearest_park = result.withColumn("row", F.row_number().over(window_spec)) \
    .filter(F.col("row") == 1) \
    .select(
        "__index_level_0__",
        F.round(F.col("distance"), 3).alias("nearest_park_distance"),  # Round to 3 decimal places
        F.col("name").alias("nearest_park_name"),
        F.col("sa2_name").alias("park_sa2_name")
    )

# Join back to the original domain data
final_result = domain.join(nearest_park, on="__index_level_0__")

# Show the result for validation
final_result.select(
    "address", 
    "property_sa2_name", 
    "extracted_price", 
    "nearest_park_distance", 
    "nearest_park_name", 
    "park_sa2_name"
) \
    .orderBy("nearest_park_distance") \
    .show(10, truncate=False)

+---------------------------------------------------+---------------------------------------------+---------------+---------------------+----------------------+-----------------------+
|address                                            |property_sa2_name                            |extracted_price|nearest_park_distance|nearest_park_name     |park_sa2_name          |
+---------------------------------------------------+---------------------------------------------+---------------+---------------------+----------------------+-----------------------+
|90 Wickhams Road, Launching Place VIC 3139         |Yarra Valley                                 |690.0          |259.975              |Yarra Valley Parklands|Warrandyte - Wonga Park|
|37 MONBULK-SEVILLE ROAD, Seville VIC 3139          |Wandin - Seville                             |738.0          |268.429              |Yarra Valley Parklands|Warrandyte - Wonga Park|
|5 Morris Court, Wandin North VIC 3139              |Wandin - Seville      

### Save

In [31]:
directory_path = "../data/curated/parkres"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

final_result.select(
    "address", 
    "property_sa2_name", 
    "extracted_price", 
    F.col("nearest_park_distance").alias("nearest_parkres_distance"),  
    F.col("nearest_park_name").alias("nearest_parkres_name"), 
    F.col("park_sa2_name").alias("parkres_sa2_name")
).write.parquet(f"{directory_path}/parkres_domain.parquet", mode="overwrite")